In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import sqlite3
%matplotlib inline
from IPython.core.display import HTML
pd.options.display.max_columns = 50
import seaborn as sb
import datetime
from sqlalchemy import create_engine
import psycopg2
HTML("<style>.container { width:100% !important; }</style>")

In [26]:
#sql_conn_local = sqlite3.connect('flask_app/data/analysed_data.sql')
#sql_cursor_local = sql_conn.cursor()
engine_AWS = create_engine('postgresql://%s:%s@forespellpostgis.cusejoju89w7.eu-west-1.rds.amazonaws.com:5432/grb_2016_03' %('kasper', 'VosseM08'))

In [35]:
user_id = "FSwWq+h3Hxn+sluX7gADjf4WcLVKpxAt"
sql_conn = create_engine('postgresql://%s:%s@forespellpostgis.cusejoju89w7.eu-west-1.rds.amazonaws.com:5432/grb_2016_03' %('kasper', 'VosseM08'))
params={'user_id':user_id}
query = '''SELECT *
                FROM feedback '''
feedback = pd.read_sql(sql = query, con=engine_AWS)
feedback.tail()

,feedback,name,session_id,time,user_id,sex,rank
1163,like,Karel,1GEwBzlwuUVt7aYmEc/nKrb7qkXwKAHz,2016-10-03 06:40:53.050305,/AuS/thid1WVjx+3wJX6eU71kmFRaEXZ,M,4.0
1164,like,Louis,1GEwBzlwuUVt7aYmEc/nKrb7qkXwKAHz,2016-10-03 06:41:19.079729,/AuS/thid1WVjx+3wJX6eU71kmFRaEXZ,M,3.0
1165,like,Louis,idR3F8crkAyZELv9dom8+0QBiRSQnOXM,2016-10-03 06:42:43.797004,RuP2Y+RmwfS6bRJG1dOXrEGTS1uVV6/U,M,1.0
1166,like,Carl,0S87TN38bR97exYTCk+dcY2K1jf5G8It,2016-10-03 07:43:29.948022,x8t72hkpCGSSNQKv+X5jlMPOkaog8V6R,F,2.0
1167,like,Pieter,0S87TN38bR97exYTCk+dcY2K1jf5G8It,2016-10-03 07:44:21.359816,x8t72hkpCGSSNQKv+X5jlMPOkaog8V6R,F,1.0


In [25]:
feedback.loc[feedback['user_id']=='/AuS/thid1WVjx+3wJX6eU71kmFRaEXZ',:]

,feedback,name,session_id,time,user_id,sex,rank
1160,like,Kasper,1GEwBzlwuUVt7aYmEc/nKrb7qkXwKAHz,2016-10-03 06:41:14.824751,/AuS/thid1WVjx+3wJX6eU71kmFRaEXZ,M,3.0
1161,like,Raymond,1GEwBzlwuUVt7aYmEc/nKrb7qkXwKAHz,2016-10-03 06:41:24.643268,/AuS/thid1WVjx+3wJX6eU71kmFRaEXZ,M,5.0
1162,like,Sander,1GEwBzlwuUVt7aYmEc/nKrb7qkXwKAHz,2016-10-03 06:41:07.838183,/AuS/thid1WVjx+3wJX6eU71kmFRaEXZ,M,1.0
1163,like,Karel,1GEwBzlwuUVt7aYmEc/nKrb7qkXwKAHz,2016-10-03 06:40:53.050305,/AuS/thid1WVjx+3wJX6eU71kmFRaEXZ,M,4.0
1164,like,Louis,1GEwBzlwuUVt7aYmEc/nKrb7qkXwKAHz,2016-10-03 06:41:19.079729,/AuS/thid1WVjx+3wJX6eU71kmFRaEXZ,M,3.0


In [59]:
feedback['rank'] = [np.nan, 1, 1,2,np.nan]
feedback

,feedback,name,session_id,time,user_id,sex,rank
8,no_like,Charlien,O1+UDVItalp5Bbj2I4YXPOFIQo3bg7oW,2016-09-28 20:26:40.927725,LyS8jqGkQaWTGm9lMrkcoffOjOH5lb7P,F,NaN
3,like,Floris,3ckDw0fIezjXUqx62utv2f1gQt0tAPmU,2016-09-28 08:46:53.124234,LyS8jqGkQaWTGm9lMrkcoffOjOH5lb7P,M,1.0
10,like,Cleo,gE6plbU54qjo/sqinvt/Xhs6rQxLqafF,2016-09-28 08:49:34.833191,LyS8jqGkQaWTGm9lMrkcoffOjOH5lb7P,F,1.0
0,like,Ensar,3ckDw0fIezjXUqx62utv2f1gQt0tAPmU,2016-09-28 08:46:44.988279,LyS8jqGkQaWTGm9lMrkcoffOjOH5lb7P,M,2.0
7,no_like,Marion,aWy20Aii1BbwdMMqpCJ44m9D9Y9q8rsz,2016-09-28 20:28:51.022527,LyS8jqGkQaWTGm9lMrkcoffOjOH5lb7P,F,NaN


In [60]:
feedback.to_sql(name='feedback', con=engine_AWS, index=False, if_exists='replace')

In [17]:
likes.loc[likes['user_id'] == 'DheKqIVxTxby4pI77THJ2lF5zfcfeOrW',:]

,feedback,name,session_id,time,user_id,sex,rank
2958,like,Léonore,j08w5bz8Lp483tgo/KEx58WQXowrJLBU,2016-09-25 10:16:22.476111,DheKqIVxTxby4pI77THJ2lF5zfcfeOrW,F,1
2972,like,Mette,j08w5bz8Lp483tgo/KEx58WQXowrJLBU,2016-09-25 10:16:36.890298,DheKqIVxTxby4pI77THJ2lF5zfcfeOrW,F,2
2998,like,Mari,j08w5bz8Lp483tgo/KEx58WQXowrJLBU,2016-09-25 10:17:07.713955,DheKqIVxTxby4pI77THJ2lF5zfcfeOrW,F,3
2999,like,June,j08w5bz8Lp483tgo/KEx58WQXowrJLBU,2016-09-25 10:17:12.505608,DheKqIVxTxby4pI77THJ2lF5zfcfeOrW,F,4
3000,like,Ella-Noor,j08w5bz8Lp483tgo/KEx58WQXowrJLBU,2016-09-25 10:17:27.359498,DheKqIVxTxby4pI77THJ2lF5zfcfeOrW,F,5


In [16]:
lookups = pd.read_sql('''SELECT * FROM name_lookups''', con=engine_AWS)
lookups = lookups.loc[lookups['region']!='',:]
lookups.to_sql(name = 'name_lookups', index=False, con = engine_AWS, if_exists = 'replace')

In [71]:
pd.read_sql('''SELECT * FROM feedback''', con=engine_AWS).tail(5)

,feedback,name,session_id,time,user_id,sex
439,like,Emile,hMErCykJ4PQyxpKyKcVJZJEajQC5oJWt,2016-09-05 11:50:11.941045,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,M
440,no_like,Richard,hMErCykJ4PQyxpKyKcVJZJEajQC5oJWt,2016-09-05 11:50:19.174217,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,M
441,no_like,Sasha,hMErCykJ4PQyxpKyKcVJZJEajQC5oJWt,2016-09-05 11:50:54.944717,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,M
442,no_like,Diego,hMErCykJ4PQyxpKyKcVJZJEajQC5oJWt,2016-09-05 11:51:12.248166,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,M
443,no_like,Alexis,hMErCykJ4PQyxpKyKcVJZJEajQC5oJWt,2016-09-05 11:51:18.306556,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,M


In [72]:
pd.read_sql('''SELECT * FROM sessions''', con=engine_AWS).tail(5)

,ip_address,session_ID,time,user_ID,user_agent,window_height,window_width
333,127.0.0.1,ttCDD+WANqAHGuhwlRURWYX+mvEz0Afk,2016-09-18 17:08:05.412191,3gG4j0QstaS/DoYZfBTQMlCOfcoo7/0G,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; ...,766,1440
334,127.0.0.1,ia4CIb19ASzCJW2m70yhuVgy/8vejcpN,2016-09-18 17:08:50.166670,gyIivrZNDOiis8mhj7J9nSHeSbeRIEOa,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3...,150,1280
335,127.0.0.1,b5cDg8b2SPAe5tLhyxMNCUeQwA3436qK,2016-09-18 17:10:03.835002,gyIivrZNDOiis8mhj7J9nSHeSbeRIEOa,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3...,391,1280
336,127.0.0.1,E0IcScWf0sQqcglXrMGgMAF6J2XhrYCT,2016-09-18 17:11:02.722404,gyIivrZNDOiis8mhj7J9nSHeSbeRIEOa,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3...,391,1280
337,127.0.0.1,JwLm6IqN9rP5P+T73l8dpX0fyJtYrF9v,2016-09-18 17:12:35.963271,gyIivrZNDOiis8mhj7J9nSHeSbeRIEOa,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3...,391,1280


In [55]:
feedback = pd.read_sql('''SELECT * FROM feedback''', con=engine_AWS)
feedback.columns = [column.lower() for column in feedback.columns]
feedback.to_sql(con=engine_AWS, name='feedback',index=False, if_exists='replace')

In [68]:
pd.read_sql('''SELECT * FROM feedback''', con=engine_AWS).tail()

,feedback,name,session_id,time,user_id,sex
439,like,Emile,hMErCykJ4PQyxpKyKcVJZJEajQC5oJWt,2016-09-05 11:50:11.941045,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,M
440,no_like,Richard,hMErCykJ4PQyxpKyKcVJZJEajQC5oJWt,2016-09-05 11:50:19.174217,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,M
441,no_like,Sasha,hMErCykJ4PQyxpKyKcVJZJEajQC5oJWt,2016-09-05 11:50:54.944717,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,M
442,no_like,Diego,hMErCykJ4PQyxpKyKcVJZJEajQC5oJWt,2016-09-05 11:51:12.248166,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,M
443,no_like,Alexis,hMErCykJ4PQyxpKyKcVJZJEajQC5oJWt,2016-09-05 11:51:18.306556,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,M


In [67]:
engine_AWS.execute('''DELETE FROM feedback WHERE time ='F' ''')